In [159]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [160]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [161]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [162]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [163]:
# !fc-cache -f

In [164]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [165]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [166]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [167]:
%%capture pwd
!pwd

In [168]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero',
 300,
 500)

In [169]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [170]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [171]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        if i['encoding'][x]['axis']:
                            for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                                if c in i['encoding'][x]['axis']:
                                    i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [172]:
df = pd.read_csv("raw/gmt_HadCRUT5.csv")
df=(df.set_index('Year')[['HadCRUT5 (degC)']]-0.35).rename(columns={'HadCRUT5 (degC)':'t'}).reset_index()

In [173]:
df

,Year,t
0,1850,-0.43
1,1851,-0.24
2,1852,-0.24
3,1853,-0.28
4,1854,-0.30
...,...,...
170,2020,0.91
171,2021,0.75
172,2022,0.79
173,2023,1.09


In [174]:
readme, f, fc = save(df,"fig1_temp",LOCAL)

,Year,t
0,1850,-0.43
1,1851,-0.24
2,1852,-0.24
3,1853,-0.28
4,1854,-0.30


In [175]:
xmin=1850
xmax=2024
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[1849, 2025],nice=False)
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Temperature anomaly in  ° C",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            # format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[1850, 2025]),
    )
)

base = alt.Chart(fc).encode(x='Year:Q')
line1=base.mark_line(color=colors['eco-light-blue'],interpolate='basis').encode(y='t:Q')
label1=line1.mark_text(text='Reference at 0: mean for 1960-90',fontSize=12,color=colors['eco-gray'],align='left',opacity=mo+0.2,dy=-75,dx=5).transform_filter("datum.Year==1850")

layer1 = (
    (xaxis+line1+label1).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_temp_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [176]:
readme, f, fc = save(df,"fig2_stripes",LOCAL)

,Year,t
0,1850,-0.43
1,1851,-0.24
2,1852,-0.24
3,1853,-0.28
4,1854,-0.30


In [177]:
stripes=['#082F6A',
'#08509C',
'#2070B4',
'#4191C6',
'#6AAED5',
'#9DCAE1',
'#C6DAEF',
'#DEEBF6',
'#FEDFD1',
'#FBBAA1',
'#FB9172',
'#FA6A4A',
'#EF3B2B',
'#CB171C',
'#A50E15',
'#66000D'
]

stripesi=['#08306B',
'#08519C',
'#2171B5',
'#4292C6',
'#6BAED6',
'#9ECAE1',
'#C6DBEF',
'#DEEBF7',
'#FEE0D2',
'#FCBBA1',
'#FC9272',
'#FB6A4A',
'#EF3B2C',
'#CB181D',
'#A50F15',
'#67000D'
]

In [178]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1860, 2021, 20))],
            titleAlign="left",
            titleAnchor="start",
            title="Deviation from 1960-1990 average world temperature",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=3).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[-0.6,0.93],
        type='quantize',
        range=stripesi
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',
              format=".1f"
  ),
        tooltip=['Year:N','Anomaly:N']).transform_calculate(Anomaly='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": 3000, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
# layer1+=(logo)
layer1=(bars1+logo).properties(height=150, width=400, title="")\
    .configure(font='Circular Std Book').configure_view(stroke=None)


if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig2_stripes_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 3

In [179]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [180]:
readme, f, fc = save(df,"fig3_gdp",LOCAL)

,Year,t
0,1990,9705.981267
1,1991,9669.67706
2,1992,9665.89026
3,1993,9675.23226
4,1994,9799.764965


In [181]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[9700,16900],
        type='quantize',
        range=stripesi
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
# layer1+=(logo)
layer1=(bars1+logo).properties(height=150, width=400, title="")\
    .configure(font='Circular Std Book').configure_view(stroke=None)


if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig3_gdp_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 4

In [182]:
co2 = [
    "#8C4614",
    "#AA5A28", #"#BE4628",
    "#C86E3C",
    "#D28250",
    "#DC9664",
    "#E6AA8C",
    "#F0C8C8",
    "#FDF0E6",
    "#F0FFDC",
    "#C3FAA0",
    "#AAF096",
    "#8CE18C",
    "#78D278",
    "#3CAA64",
    "#1E8732",
    "#006400"
]

In [183]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [184]:
readme, f, fc = save(df,"fig4_co2",LOCAL)

,Year,t
0,1990,4.020868
1,1991,3.983343
2,1992,3.91031
3,1993,3.874985
4,1994,3.842027


In [185]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        # domain=[9700,16900],
        type='quantize',
        range=co2,
        reverse=True
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format=".1f"),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
# layer1+=(logo)
layer1=(bars1+logo).properties(height=150, width=400, title="")\
    .configure(font='Circular Std Book').configure_view(stroke=None)


if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig4_co2_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 5

In [186]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [187]:
readme, f, fc = save(df,"fig5_gpr",LOCAL)

,Year,t
0,1990,2413.902178
1,1991,2427.528167
2,1992,2471.89853
3,1993,2496.843635
4,1994,2550.67551


In [188]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        # domain=[2600,3800],
        type='quantize',
        range=co2
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format=",.0f"),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')

layer1 = (
    (bars1+bars1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
# layer1+=(logo)
layer1=(bars1+logo).properties(height=150, width=400, title="")\
    .configure(font='Circular Std Book').configure_view(stroke=None)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig5_gpr_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 6

## a

In [189]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
# df = pd.read_excel("raw/API_NY.GDP.PCAP.CD_DS2_en_excel_v2_5607109.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [190]:
readme, f, fc = save(df,"fig6a_gdp_world",LOCAL)

,Year,t
0,1990,9705.981267
1,1991,9669.67706
2,1992,9665.89026
3,1993,9675.23226
4,1994,9799.764965


In [191]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[436,157602],
        type='quantize',
        range=stripesi
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')
label1=bars1.mark_text(text='World',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (label1+bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6a_gdp_world_local



alt.LayerChart(...)

alt.LayerChart(...)


## b

In [192]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [193]:
readme, f, fc = save(df,"fig6b_gdp_ch",LOCAL)

,Year,t
0,1990,56232.02608
1,1991,55025.011308
2,1992,54397.880761
3,1993,53836.808853
4,1994,54087.519552


In [194]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[436,157602],
        type='quantize',
        range=stripesi
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')
label1=bars1.mark_text(text='Switzerland',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (label1+bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6b_gdp_ch_local



alt.LayerChart(...)

alt.LayerChart(...)

## c

In [195]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [196]:
readme, f, fc = save(df,"fig6b_gdp_mz",LOCAL)

,Year,t
0,1990,460.123697
1,1991,473.612004
2,1992,436.376416
3,1993,469.397817
4,1994,476.370759


In [197]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="GDP per capita at PPP in 2017 international constant US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=12).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[436,157602],
        type='quantize',
        range=stripesi
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GDP:N']).transform_calculate(GDP='round(datum.t*100)/100')
label1=bars1.mark_text(text='Mozambique',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (label1+bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6b_gdp_mz_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 7

## a

In [198]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [199]:
readme, f, fc = save(df,"fig7_co2_world",LOCAL)

,Year,t
0,1990,4.020868
1,1991,3.983343
2,1992,3.91031
3,1993,3.874985
4,1994,3.842027


In [200]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[0.001,47.65696201],
        type='quantize',
        range=co2,
        reverse=True
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')
label1=bars1.mark_text(text='World',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig7_co2_world_local



alt.LayerChart(...)

alt.LayerChart(...)

## b

In [201]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [202]:
readme, f, fc = save(df,"fig7b_co2_ch",LOCAL)

,Year,t
0,1990,6.451698
1,1991,6.715728
2,1992,6.647299
3,1993,6.262661
4,1994,6.090113


In [203]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[0.001,47.65696201],
        type='quantize',
        range=co2,
        reverse=True
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')
label1=bars1.mark_text(text='Switzerland',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig7b_co2_ch_local



alt.LayerChart(...)

alt.LayerChart(...)

## c

In [204]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [205]:
readme, f, fc = save(df,"fig7c_co2_mz",LOCAL)

,Year,t
0,1990,0.08398
1,1991,0.069027
2,1992,0.079423
3,1993,0.090705
4,1994,0.073553


In [206]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Carbon dioxide emissions per capita in metric tons",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[0.001,47.65696201],
        type='quantize',
        range=co2,
        reverse=True
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','Emissions:N']).transform_calculate(Emissions='round(datum.t*100)/100')
label1=bars1.mark_text(text='Mozambique',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig7c_co2_mz_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 8

## a

In [207]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='World'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [208]:
readme, f, fc = save(df,"fig8a_gpr_world",LOCAL)

,Year,t
0,1990,2413.902178
1,1991,2427.528167
2,1992,2471.89853
3,1993,2496.843635
4,1994,2550.67551


In [209]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,120000],
        type='quantize',
        range=co2
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')
label1=bars1.mark_text(text='World',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8a_gpr_world_local



alt.LayerChart(...)

alt.LayerChart(...)

## b

In [210]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='Switzerland'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [211]:
readme, f, fc = save(df,"fig8b_gpr_ch",LOCAL)

,Year,t
0,1990,8715.849181
1,1991,8193.454903
2,1992,8183.456325
3,1993,8596.475342
4,1994,8881.201695


In [212]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,120000],
        type='quantize',
        range=co2
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')
label1=bars1.mark_text(text='Switzerland',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8b_gpr_ch_local



alt.LayerChart(...)

alt.LayerChart(...)

## c

In [213]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df1=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df1.columns=['t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df2=df[df['Country Name']=='Mozambique'].drop(['Country Name',
                                      'Country Code','Indicator Name','Indicator Code'],axis=1).T.dropna()
df2.columns=['t']

df=(df1/df2).dropna().reset_index()
df.columns=['Year','t']
df['Year']=df['Year'].astype(int)

In [214]:
readme, f, fc = save(df,"fig8c_gpr_mz",LOCAL)

,Year,t
0,1990,5478.940535
1,1991,6861.236916
2,1992,5494.323758
3,1993,5175.015014
4,1994,6476.603298


In [215]:
xmin=1850
xmax=2024
xaxis = base = alt.Chart(fc).encode(
    alt.X(
        "Year:O",
        # sort=[],
        axis=alt.Axis(
            grid=False,
            values=[str(i) for i in list(range(1990, 2025, 5))],
            titleAlign="left",
            titleAnchor="start",
            title="Greening prosperity per capita in discounted 2017 US dollars",
            titleY=-168,
            titleX=0,
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domain=False,
            # domainColor=colors["eco-gray"],
            # tickCount=10,
            # domainOpacity=mo,
            # tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            # orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1849, 2025],nice=False)
    )
)

bars1=base.mark_bar(stroke=None,width=13).encode(color=alt.Color('t:Q',scale=alt.Scale(
        domain=[1000,120000],
        type='quantize',
        range=co2
    )).legend(title="",labelColor=colors["eco-gray"],gradientLength=150,type='gradient',format="s"),
        tooltip=['Year:N','GPR:N']).transform_calculate(GPR='round(datum.t*100)/100')
label1=bars1.mark_text(text='Mozambique',fill='white',fontSize=12,align='left',dy=-60).transform_filter('datum.Year==1990')

layer1 = (
    (bars1+bars1+label1).properties(height=150, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 1.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:O',y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8c_gpr_mz_local



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 9

## a

In [216]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df=df.loc[df.index.dropna()].reset_index()
df.columns=['Country','t']

In [217]:
readme, f, fc = save(df,"fig9a_gdp_map",LOCAL)

,Country,t
0,ZWE,1990.319419
1,ZMB,3183.650773
2,YEM,1.000000
3,WLD,16204.169107
4,PSE,5402.538773


In [218]:
# countries = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 'features')
countries = alt.Data(url='https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 
                     format=alt.DataFormat(property='features',type='json'))

layer1=alt.Chart(countries).mark_geoshape().encode(
        color=alt.condition('datum.t == 1', alt.value('lightgray'),
        alt.Color('t:Q', 
        scale=alt.Scale(
        domain=[711.3552634,111751.3148],
        type='quantize',
        range=stripesi
    ))).legend(title="",labelColor=colors["eco-gray"],legendY=120,gradientLength=150,type='gradient',format="s"),
        tooltip=['Country:N','GDP:N']).transform_lookup(
    lookup='properties.ISO3',
    from_=alt.LookupData(df, 'Country', ['Country','t'])
).project(
    type='naturalEarth1'
).transform_calculate(GDP='round(datum.t*100)/100')

logo=alt.Chart(pd.DataFrame([{"x": 0, "y": 0, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-160,opacity=mo,xOffset=500).encode(x=alt.X('x:O',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)

layer1+=(logo)

layer1=layer1.properties(height=300, width=500, title={'text':"GDP per capita at PPP in 2020, in 2017 international constant US dollars",
                                                'color':'#9597AA'
                                                })\
    .configure(font='Circular Std Book').configure_view(stroke=None)
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
        
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()

fig9a_gdp_map_local



alt.LayerChart(...)

## b

In [219]:
df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df=df.loc[df.index.dropna()].reset_index()
df.columns=['Country','t']

In [220]:
readme, f, fc = save(df,"fig9b_co2_map",LOCAL)

,Country,t
0,ABW,1.000000
1,AFE,0.795420
2,AFG,0.223479
3,AFW,0.463150
4,AGO,0.592743


In [221]:
# countries = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 'features')
countries = alt.Data(url='https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 
                     format=alt.DataFormat(property='features',type='json'))

layer1=alt.Chart(countries).mark_geoshape().encode(
        color=alt.condition('datum.t == 1', alt.value('lightgray'),
        alt.Color('t:Q', 
        scale=alt.Scale(
        domain=[0.032585,31.72684],
        type='quantize',
        range=co2,
            reverse=True
    ))).legend(title="",labelColor=colors["eco-gray"],legendY=120,gradientLength=150,type='gradient',format="s"),
        tooltip=['Country:N','Emissions:N']).transform_lookup(
    lookup='properties.ISO3',
    from_=alt.LookupData(df, 'Country', ['Country','t'])
).project(
    type='naturalEarth1'
).transform_calculate(Emissions='round(datum.t*100)/100')

logo=alt.Chart(pd.DataFrame([{"x": 0, "y": 0, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-160,opacity=mo,xOffset=500).encode(x=alt.X('x:O',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1=layer1.properties(height=300, width=500, title={'text':"Carbon dioxide emissions per capita in 2020 in metric tons",
                                                'color':'#9597AA'
                                                })\
    .configure(font='Circular Std Book').configure_view(stroke=None)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
        
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()

fig9b_co2_map_local



alt.LayerChart(...)

## c

In [222]:
df = pd.read_excel("raw/API_NY.GDP.PCAP.PP.KD_DS2_en_excel_v2_5607670.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df1=df.loc[df.index.dropna()].reset_index()
df1.columns=['Country','t']

df = pd.read_excel("raw/API_EN.ATM.CO2E.PC_DS2_en_excel_v2_5607813.xlsx",skiprows=3)
df=df.set_index('Country Code').drop(['Country Name',
                                     'Indicator Name','Indicator Code'],axis=1)['2020'].dropna()
df2=df.loc[df.index.dropna()].reset_index()
df2.columns=['Country','t']

df=(df1.set_index('Country').replace(1,np.nan)/df2.set_index('Country')).fillna(1).reset_index()

In [223]:
df

,Country,t
0,ABW,33155.243239
1,AFE,4343.646774
2,AFG,8807.727477
3,AFW,8545.679518
4,AGO,10172.523712
...,...,...
261,XKX,10706.513976
262,YEM,1.000000
263,ZAF,1916.379600
264,ZMB,7921.446346


In [224]:
readme, f, fc = save(df,"fig9c_gpr_map",LOCAL)

,Country,t
0,ABW,33155.243239
1,AFE,4343.646774
2,AFG,8807.727477
3,AFW,8545.679518
4,AGO,10172.523712


In [225]:
# countries = alt.topo_feature('https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 'features')
countries = alt.Data(url='https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/e4e/ch1_GDP/data/chart1_4_world_map_gdp_world_countries.geojson', 
                     format=alt.DataFormat(property='features',type='json'))

layer1=alt.Chart(countries).mark_geoshape().encode(
        color=alt.condition('datum.t == 1', alt.value('lightgray'),
        alt.Color('t:Q', 
        scale=alt.Scale(
        domain=[1401.170474,32041.68286],
        type='quantize',
        range=co2
    ))).legend(title="",labelColor=colors["eco-gray"],legendY=120,gradientLength=150,type='gradient',format="s"),
        tooltip=['Country:N','GPR:N']).transform_lookup(
    lookup='properties.ISO3',
    from_=alt.LookupData(df, 'Country', ['Country','t'])
).project(
    type='naturalEarth1'
).transform_calculate(GPR='round(datum.t*100)/100')

logo=alt.Chart(pd.DataFrame([{"x": 0, "y": 0, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-160,opacity=mo,xOffset=500).encode(x=alt.X('x:O',axis=None),y=alt.Y('y:Q',axis=None),url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)

layer1+=(logo)

layer1=layer1.properties(height=300, width=500, title={'text':"Greening prosperity per capita in 2020 in discounted 2017 US dollars",
                                                'color':'#9597AA'
                                                })\
    .configure(font='Circular Std Book').configure_view(stroke=None)

if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
        
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()

fig9c_gpr_map_local



alt.LayerChart(...)

# Post-process

## Clean up `SVG`s
`base64 URI` encode images

In [226]:
from base64 import b64encode

In [227]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [228]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

fig1_temp_local.svg
fig1_temp_local_dark.svg
fig2_stripes_local.svg
fig2_stripes_local_dark.svg
fig3_gdp_local.svg
fig3_gdp_local_dark.svg
fig4_co2_local.svg
fig4_co2_local_dark.svg
fig5_gpr_local.svg
fig5_gpr_local_dark.svg
fig6a_gdp_world_local.svg
fig6a_gdp_world_local_dark.svg
fig6b_gdp_ch_local.svg
fig6b_gdp_ch_local_dark.svg
fig6b_gdp_mz_local.svg
fig6b_gdp_mz_local_dark.svg
fig7b_co2_ch_local.svg
fig7b_co2_ch_local_dark.svg
fig7c_co2_mz_local.svg
fig7c_co2_mz_local_dark.svg
fig7_co2_world_local.svg
fig7_co2_world_local_dark.svg
fig8a_gpr_world_local.svg
fig8a_gpr_world_local_dark.svg
fig8b_gpr_ch_local.svg
fig8b_gpr_ch_local_dark.svg
fig8c_gpr_mz_local.svg
fig8c_gpr_mz_local_dark.svg
fig9a_gdp_map_local.svg
fig9a_gdp_map_local_no_branding.svg
fig9b_co2_map_local.svg
fig9b_co2_map_local_no_branding.svg
fig9c_gpr_map_local.svg
fig9c_gpr_map_local_no_branding.svg


## README

In [229]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_temp_local.csv)  
GitHub: [fig1_temp_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_local.png) [`svg`](visualisation/fig1_temp_local_local.svg) [`json`](visualisation/fig1_temp_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_local.png) [`svg`](visualisation/fig1_temp_local.svg) [`json`](visualisation/fig1_temp_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_local_no_branding.png) [`svg`](visualisation/fig1_temp_local_local_no_branding.svg) [`json`](visualisation/fig1_temp_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_local_no_branding.png) [`svg`](visualisation/fig1_temp_local_no_branding.svg) [`json`](visualisation/fig1_temp_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_local_dark.png) [`svg`](visualisation/fig1_temp_local_local_dark.svg) [`json`](visualisation/fig1_temp_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_local_dark.png) [`svg`](visualisation/fig1_temp_local_dark.svg) [`json`](visualisation/fig1_temp_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_temp_local_local_no_branding_dark.png) [`svg`](visualisation/fig1_temp_local_local_no_branding_dark.svg) [`json`](visualisation/fig1_temp_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_temp_local_no_branding_dark.png) [`svg`](visualisation/fig1_temp_local_no_branding_dark.svg) [`json`](visualisation/fig1_temp_local_no_branding_dark.json)   

!["fig1_temp_local"](visualisation/fig1_temp_local.svg "fig1_temp_local")

  

!["fig1_temp_local_dark"](visualisation/fig1_temp_local_dark.svg "fig1_temp_local")

## Figure 2  

Data: [`csv`](data/fig2_stripes_local.csv)  
GitHub: [fig2_stripes_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_local.png) [`svg`](visualisation/fig2_stripes_local_local.svg) [`json`](visualisation/fig2_stripes_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_local.png) [`svg`](visualisation/fig2_stripes_local.svg) [`json`](visualisation/fig2_stripes_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_local_no_branding.png) [`svg`](visualisation/fig2_stripes_local_local_no_branding.svg) [`json`](visualisation/fig2_stripes_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_local_no_branding.png) [`svg`](visualisation/fig2_stripes_local_no_branding.svg) [`json`](visualisation/fig2_stripes_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_local_dark.png) [`svg`](visualisation/fig2_stripes_local_local_dark.svg) [`json`](visualisation/fig2_stripes_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_local_dark.png) [`svg`](visualisation/fig2_stripes_local_dark.svg) [`json`](visualisation/fig2_stripes_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_stripes_local_local_no_branding_dark.png) [`svg`](visualisation/fig2_stripes_local_local_no_branding_dark.svg) [`json`](visualisation/fig2_stripes_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_stripes_local_no_branding_dark.png) [`svg`](visualisation/fig2_stripes_local_no_branding_dark.svg) [`json`](visualisation/fig2_stripes_local_no_branding_dark.json)   

!["fig2_stripes_local"](visualisation/fig2_stripes_local.svg "fig2_stripes_local")

  

!["fig2_stripes_local_dark"](visualisation/fig2_stripes_local_dark.svg "fig2_stripes_local")

## Figure 3  

Data: [`csv`](data/fig3_gdp_local.csv)  
GitHub: [fig3_gdp_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_local.png) [`svg`](visualisation/fig3_gdp_local_local.svg) [`json`](visualisation/fig3_gdp_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_local.png) [`svg`](visualisation/fig3_gdp_local.svg) [`json`](visualisation/fig3_gdp_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_local_no_branding.png) [`svg`](visualisation/fig3_gdp_local_local_no_branding.svg) [`json`](visualisation/fig3_gdp_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_local_no_branding.png) [`svg`](visualisation/fig3_gdp_local_no_branding.svg) [`json`](visualisation/fig3_gdp_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_local_dark.png) [`svg`](visualisation/fig3_gdp_local_local_dark.svg) [`json`](visualisation/fig3_gdp_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_local_dark.png) [`svg`](visualisation/fig3_gdp_local_dark.svg) [`json`](visualisation/fig3_gdp_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_gdp_local_local_no_branding_dark.png) [`svg`](visualisation/fig3_gdp_local_local_no_branding_dark.svg) [`json`](visualisation/fig3_gdp_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_gdp_local_no_branding_dark.png) [`svg`](visualisation/fig3_gdp_local_no_branding_dark.svg) [`json`](visualisation/fig3_gdp_local_no_branding_dark.json)   

!["fig3_gdp_local"](visualisation/fig3_gdp_local.svg "fig3_gdp_local")

  

!["fig3_gdp_local_dark"](visualisation/fig3_gdp_local_dark.svg "fig3_gdp_local")

## Figure 4  

Data: [`csv`](data/fig4_co2_local.csv)  
GitHub: [fig4_co2_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_local.png) [`svg`](visualisation/fig4_co2_local_local.svg) [`json`](visualisation/fig4_co2_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_local.png) [`svg`](visualisation/fig4_co2_local.svg) [`json`](visualisation/fig4_co2_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_local_no_branding.png) [`svg`](visualisation/fig4_co2_local_local_no_branding.svg) [`json`](visualisation/fig4_co2_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_local_no_branding.png) [`svg`](visualisation/fig4_co2_local_no_branding.svg) [`json`](visualisation/fig4_co2_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_local_dark.png) [`svg`](visualisation/fig4_co2_local_local_dark.svg) [`json`](visualisation/fig4_co2_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_local_dark.png) [`svg`](visualisation/fig4_co2_local_dark.svg) [`json`](visualisation/fig4_co2_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_co2_local_local_no_branding_dark.png) [`svg`](visualisation/fig4_co2_local_local_no_branding_dark.svg) [`json`](visualisation/fig4_co2_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_co2_local_no_branding_dark.png) [`svg`](visualisation/fig4_co2_local_no_branding_dark.svg) [`json`](visualisation/fig4_co2_local_no_branding_dark.json)   

!["fig4_co2_local"](visualisation/fig4_co2_local.svg "fig4_co2_local")

  

!["fig4_co2_local_dark"](visualisation/fig4_co2_local_dark.svg "fig4_co2_local")

## Figure 5  

Data: [`csv`](data/fig5_gpr_local.csv)  
GitHub: [fig5_gpr_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_local.png) [`svg`](visualisation/fig5_gpr_local_local.svg) [`json`](visualisation/fig5_gpr_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_local.png) [`svg`](visualisation/fig5_gpr_local.svg) [`json`](visualisation/fig5_gpr_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_local_no_branding.png) [`svg`](visualisation/fig5_gpr_local_local_no_branding.svg) [`json`](visualisation/fig5_gpr_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_local_no_branding.png) [`svg`](visualisation/fig5_gpr_local_no_branding.svg) [`json`](visualisation/fig5_gpr_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_local_dark.png) [`svg`](visualisation/fig5_gpr_local_local_dark.svg) [`json`](visualisation/fig5_gpr_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_local_dark.png) [`svg`](visualisation/fig5_gpr_local_dark.svg) [`json`](visualisation/fig5_gpr_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_gpr_local_local_no_branding_dark.png) [`svg`](visualisation/fig5_gpr_local_local_no_branding_dark.svg) [`json`](visualisation/fig5_gpr_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_gpr_local_no_branding_dark.png) [`svg`](visualisation/fig5_gpr_local_no_branding_dark.svg) [`json`](visualisation/fig5_gpr_local_no_branding_dark.json)   

!["fig5_gpr_local"](visualisation/fig5_gpr_local.svg "fig5_gpr_local")

  

!["fig5_gpr_local_dark"](visualisation/fig5_gpr_local_dark.svg "fig5_gpr_local")

## Figure 6a  

Data: [`csv`](data/fig6a_gdp_world_local.csv)  
GitHub: [fig6a_gdp_world_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_local.png) [`svg`](visualisation/fig6a_gdp_world_local_local.svg) [`json`](visualisation/fig6a_gdp_world_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_local.png) [`svg`](visualisation/fig6a_gdp_world_local.svg) [`json`](visualisation/fig6a_gdp_world_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_local_no_branding.png) [`svg`](visualisation/fig6a_gdp_world_local_local_no_branding.svg) [`json`](visualisation/fig6a_gdp_world_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_local_no_branding.png) [`svg`](visualisation/fig6a_gdp_world_local_no_branding.svg) [`json`](visualisation/fig6a_gdp_world_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_local_dark.png) [`svg`](visualisation/fig6a_gdp_world_local_local_dark.svg) [`json`](visualisation/fig6a_gdp_world_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_local_dark.png) [`svg`](visualisation/fig6a_gdp_world_local_dark.svg) [`json`](visualisation/fig6a_gdp_world_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6a_gdp_world_local_local_no_branding_dark.png) [`svg`](visualisation/fig6a_gdp_world_local_local_no_branding_dark.svg) [`json`](visualisation/fig6a_gdp_world_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6a_gdp_world_local_no_branding_dark.png) [`svg`](visualisation/fig6a_gdp_world_local_no_branding_dark.svg) [`json`](visualisation/fig6a_gdp_world_local_no_branding_dark.json)   

!["fig6a_gdp_world_local"](visualisation/fig6a_gdp_world_local.svg "fig6a_gdp_world_local")

  

!["fig6a_gdp_world_local_dark"](visualisation/fig6a_gdp_world_local_dark.svg "fig6a_gdp_world_local")

## Figure 6b  

Data: [`csv`](data/fig6b_gdp_ch_local.csv)  
GitHub: [fig6b_gdp_ch_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_local.png) [`svg`](visualisation/fig6b_gdp_ch_local_local.svg) [`json`](visualisation/fig6b_gdp_ch_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_local.png) [`svg`](visualisation/fig6b_gdp_ch_local.svg) [`json`](visualisation/fig6b_gdp_ch_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_local_no_branding.png) [`svg`](visualisation/fig6b_gdp_ch_local_local_no_branding.svg) [`json`](visualisation/fig6b_gdp_ch_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_local_no_branding.png) [`svg`](visualisation/fig6b_gdp_ch_local_no_branding.svg) [`json`](visualisation/fig6b_gdp_ch_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_local_dark.png) [`svg`](visualisation/fig6b_gdp_ch_local_local_dark.svg) [`json`](visualisation/fig6b_gdp_ch_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_local_dark.png) [`svg`](visualisation/fig6b_gdp_ch_local_dark.svg) [`json`](visualisation/fig6b_gdp_ch_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_ch_local_local_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_ch_local_local_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_ch_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_ch_local_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_ch_local_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_ch_local_no_branding_dark.json)   

!["fig6b_gdp_ch_local"](visualisation/fig6b_gdp_ch_local.svg "fig6b_gdp_ch_local")

  

!["fig6b_gdp_ch_local_dark"](visualisation/fig6b_gdp_ch_local_dark.svg "fig6b_gdp_ch_local")

## Figure 6b  

Data: [`csv`](data/fig6b_gdp_mz_local.csv)  
GitHub: [fig6b_gdp_mz_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_local.png) [`svg`](visualisation/fig6b_gdp_mz_local_local.svg) [`json`](visualisation/fig6b_gdp_mz_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_local.png) [`svg`](visualisation/fig6b_gdp_mz_local.svg) [`json`](visualisation/fig6b_gdp_mz_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_local_no_branding.png) [`svg`](visualisation/fig6b_gdp_mz_local_local_no_branding.svg) [`json`](visualisation/fig6b_gdp_mz_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_local_no_branding.png) [`svg`](visualisation/fig6b_gdp_mz_local_no_branding.svg) [`json`](visualisation/fig6b_gdp_mz_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_local_dark.png) [`svg`](visualisation/fig6b_gdp_mz_local_local_dark.svg) [`json`](visualisation/fig6b_gdp_mz_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_local_dark.png) [`svg`](visualisation/fig6b_gdp_mz_local_dark.svg) [`json`](visualisation/fig6b_gdp_mz_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6b_gdp_mz_local_local_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_mz_local_local_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_mz_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6b_gdp_mz_local_no_branding_dark.png) [`svg`](visualisation/fig6b_gdp_mz_local_no_branding_dark.svg) [`json`](visualisation/fig6b_gdp_mz_local_no_branding_dark.json)   

!["fig6b_gdp_mz_local"](visualisation/fig6b_gdp_mz_local.svg "fig6b_gdp_mz_local")

  

!["fig6b_gdp_mz_local_dark"](visualisation/fig6b_gdp_mz_local_dark.svg "fig6b_gdp_mz_local")

## Figure 7  

Data: [`csv`](data/fig7_co2_world_local.csv)  
GitHub: [fig7_co2_world_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_local.png) [`svg`](visualisation/fig7_co2_world_local_local.svg) [`json`](visualisation/fig7_co2_world_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_local.png) [`svg`](visualisation/fig7_co2_world_local.svg) [`json`](visualisation/fig7_co2_world_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_local_no_branding.png) [`svg`](visualisation/fig7_co2_world_local_local_no_branding.svg) [`json`](visualisation/fig7_co2_world_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_local_no_branding.png) [`svg`](visualisation/fig7_co2_world_local_no_branding.svg) [`json`](visualisation/fig7_co2_world_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_local_dark.png) [`svg`](visualisation/fig7_co2_world_local_local_dark.svg) [`json`](visualisation/fig7_co2_world_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_local_dark.png) [`svg`](visualisation/fig7_co2_world_local_dark.svg) [`json`](visualisation/fig7_co2_world_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7_co2_world_local_local_no_branding_dark.png) [`svg`](visualisation/fig7_co2_world_local_local_no_branding_dark.svg) [`json`](visualisation/fig7_co2_world_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7_co2_world_local_no_branding_dark.png) [`svg`](visualisation/fig7_co2_world_local_no_branding_dark.svg) [`json`](visualisation/fig7_co2_world_local_no_branding_dark.json)   

!["fig7_co2_world_local"](visualisation/fig7_co2_world_local.svg "fig7_co2_world_local")

  

!["fig7_co2_world_local_dark"](visualisation/fig7_co2_world_local_dark.svg "fig7_co2_world_local")

## Figure 7b  

Data: [`csv`](data/fig7b_co2_ch_local.csv)  
GitHub: [fig7b_co2_ch_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_local.png) [`svg`](visualisation/fig7b_co2_ch_local_local.svg) [`json`](visualisation/fig7b_co2_ch_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_local.png) [`svg`](visualisation/fig7b_co2_ch_local.svg) [`json`](visualisation/fig7b_co2_ch_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_local_no_branding.png) [`svg`](visualisation/fig7b_co2_ch_local_local_no_branding.svg) [`json`](visualisation/fig7b_co2_ch_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_local_no_branding.png) [`svg`](visualisation/fig7b_co2_ch_local_no_branding.svg) [`json`](visualisation/fig7b_co2_ch_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_local_dark.png) [`svg`](visualisation/fig7b_co2_ch_local_local_dark.svg) [`json`](visualisation/fig7b_co2_ch_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_local_dark.png) [`svg`](visualisation/fig7b_co2_ch_local_dark.svg) [`json`](visualisation/fig7b_co2_ch_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7b_co2_ch_local_local_no_branding_dark.png) [`svg`](visualisation/fig7b_co2_ch_local_local_no_branding_dark.svg) [`json`](visualisation/fig7b_co2_ch_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7b_co2_ch_local_no_branding_dark.png) [`svg`](visualisation/fig7b_co2_ch_local_no_branding_dark.svg) [`json`](visualisation/fig7b_co2_ch_local_no_branding_dark.json)   

!["fig7b_co2_ch_local"](visualisation/fig7b_co2_ch_local.svg "fig7b_co2_ch_local")

  

!["fig7b_co2_ch_local_dark"](visualisation/fig7b_co2_ch_local_dark.svg "fig7b_co2_ch_local")

## Figure 7c  

Data: [`csv`](data/fig7c_co2_mz_local.csv)  
GitHub: [fig7c_co2_mz_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_local.png) [`svg`](visualisation/fig7c_co2_mz_local_local.svg) [`json`](visualisation/fig7c_co2_mz_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_local.png) [`svg`](visualisation/fig7c_co2_mz_local.svg) [`json`](visualisation/fig7c_co2_mz_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_local_no_branding.png) [`svg`](visualisation/fig7c_co2_mz_local_local_no_branding.svg) [`json`](visualisation/fig7c_co2_mz_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_local_no_branding.png) [`svg`](visualisation/fig7c_co2_mz_local_no_branding.svg) [`json`](visualisation/fig7c_co2_mz_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_local_dark.png) [`svg`](visualisation/fig7c_co2_mz_local_local_dark.svg) [`json`](visualisation/fig7c_co2_mz_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_local_dark.png) [`svg`](visualisation/fig7c_co2_mz_local_dark.svg) [`json`](visualisation/fig7c_co2_mz_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig7c_co2_mz_local_local_no_branding_dark.png) [`svg`](visualisation/fig7c_co2_mz_local_local_no_branding_dark.svg) [`json`](visualisation/fig7c_co2_mz_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig7c_co2_mz_local_no_branding_dark.png) [`svg`](visualisation/fig7c_co2_mz_local_no_branding_dark.svg) [`json`](visualisation/fig7c_co2_mz_local_no_branding_dark.json)   

!["fig7c_co2_mz_local"](visualisation/fig7c_co2_mz_local.svg "fig7c_co2_mz_local")

  

!["fig7c_co2_mz_local_dark"](visualisation/fig7c_co2_mz_local_dark.svg "fig7c_co2_mz_local")

## Figure 8a  

Data: [`csv`](data/fig8a_gpr_world_local.csv)  
GitHub: [fig8a_gpr_world_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_local.png) [`svg`](visualisation/fig8a_gpr_world_local_local.svg) [`json`](visualisation/fig8a_gpr_world_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_local.png) [`svg`](visualisation/fig8a_gpr_world_local.svg) [`json`](visualisation/fig8a_gpr_world_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_local_no_branding.png) [`svg`](visualisation/fig8a_gpr_world_local_local_no_branding.svg) [`json`](visualisation/fig8a_gpr_world_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_local_no_branding.png) [`svg`](visualisation/fig8a_gpr_world_local_no_branding.svg) [`json`](visualisation/fig8a_gpr_world_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_local_dark.png) [`svg`](visualisation/fig8a_gpr_world_local_local_dark.svg) [`json`](visualisation/fig8a_gpr_world_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_local_dark.png) [`svg`](visualisation/fig8a_gpr_world_local_dark.svg) [`json`](visualisation/fig8a_gpr_world_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8a_gpr_world_local_local_no_branding_dark.png) [`svg`](visualisation/fig8a_gpr_world_local_local_no_branding_dark.svg) [`json`](visualisation/fig8a_gpr_world_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8a_gpr_world_local_no_branding_dark.png) [`svg`](visualisation/fig8a_gpr_world_local_no_branding_dark.svg) [`json`](visualisation/fig8a_gpr_world_local_no_branding_dark.json)   

!["fig8a_gpr_world_local"](visualisation/fig8a_gpr_world_local.svg "fig8a_gpr_world_local")

  

!["fig8a_gpr_world_local_dark"](visualisation/fig8a_gpr_world_local_dark.svg "fig8a_gpr_world_local")

## Figure 8b  

Data: [`csv`](data/fig8b_gpr_ch_local.csv)  
GitHub: [fig8b_gpr_ch_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_local.png) [`svg`](visualisation/fig8b_gpr_ch_local_local.svg) [`json`](visualisation/fig8b_gpr_ch_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_local.png) [`svg`](visualisation/fig8b_gpr_ch_local.svg) [`json`](visualisation/fig8b_gpr_ch_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_local_no_branding.png) [`svg`](visualisation/fig8b_gpr_ch_local_local_no_branding.svg) [`json`](visualisation/fig8b_gpr_ch_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_local_no_branding.png) [`svg`](visualisation/fig8b_gpr_ch_local_no_branding.svg) [`json`](visualisation/fig8b_gpr_ch_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_local_dark.png) [`svg`](visualisation/fig8b_gpr_ch_local_local_dark.svg) [`json`](visualisation/fig8b_gpr_ch_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_local_dark.png) [`svg`](visualisation/fig8b_gpr_ch_local_dark.svg) [`json`](visualisation/fig8b_gpr_ch_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8b_gpr_ch_local_local_no_branding_dark.png) [`svg`](visualisation/fig8b_gpr_ch_local_local_no_branding_dark.svg) [`json`](visualisation/fig8b_gpr_ch_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8b_gpr_ch_local_no_branding_dark.png) [`svg`](visualisation/fig8b_gpr_ch_local_no_branding_dark.svg) [`json`](visualisation/fig8b_gpr_ch_local_no_branding_dark.json)   

!["fig8b_gpr_ch_local"](visualisation/fig8b_gpr_ch_local.svg "fig8b_gpr_ch_local")

  

!["fig8b_gpr_ch_local_dark"](visualisation/fig8b_gpr_ch_local_dark.svg "fig8b_gpr_ch_local")

## Figure 8c  

Data: [`csv`](data/fig8c_gpr_mz_local.csv)  
GitHub: [fig8c_gpr_mz_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_local.png) [`svg`](visualisation/fig8c_gpr_mz_local_local.svg) [`json`](visualisation/fig8c_gpr_mz_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_local.png) [`svg`](visualisation/fig8c_gpr_mz_local.svg) [`json`](visualisation/fig8c_gpr_mz_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_local_no_branding.png) [`svg`](visualisation/fig8c_gpr_mz_local_local_no_branding.svg) [`json`](visualisation/fig8c_gpr_mz_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_local_no_branding.png) [`svg`](visualisation/fig8c_gpr_mz_local_no_branding.svg) [`json`](visualisation/fig8c_gpr_mz_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_local_dark.png) [`svg`](visualisation/fig8c_gpr_mz_local_local_dark.svg) [`json`](visualisation/fig8c_gpr_mz_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_local_dark.png) [`svg`](visualisation/fig8c_gpr_mz_local_dark.svg) [`json`](visualisation/fig8c_gpr_mz_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8c_gpr_mz_local_local_no_branding_dark.png) [`svg`](visualisation/fig8c_gpr_mz_local_local_no_branding_dark.svg) [`json`](visualisation/fig8c_gpr_mz_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8c_gpr_mz_local_no_branding_dark.png) [`svg`](visualisation/fig8c_gpr_mz_local_no_branding_dark.svg) [`json`](visualisation/fig8c_gpr_mz_local_no_branding_dark.json)   

!["fig8c_gpr_mz_local"](visualisation/fig8c_gpr_mz_local.svg "fig8c_gpr_mz_local")

  

!["fig8c_gpr_mz_local_dark"](visualisation/fig8c_gpr_mz_local_dark.svg "fig8c_gpr_mz_local")

## Figure 9a  

Data: [`csv`](data/fig9a_gdp_map_local.csv)  
GitHub: [fig9a_gdp_map_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_local.png) [`svg`](visualisation/fig9a_gdp_map_local_local.svg) [`json`](visualisation/fig9a_gdp_map_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_local.png) [`svg`](visualisation/fig9a_gdp_map_local.svg) [`json`](visualisation/fig9a_gdp_map_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_local_no_branding.png) [`svg`](visualisation/fig9a_gdp_map_local_local_no_branding.svg) [`json`](visualisation/fig9a_gdp_map_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_local_no_branding.png) [`svg`](visualisation/fig9a_gdp_map_local_no_branding.svg) [`json`](visualisation/fig9a_gdp_map_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_local_dark.png) [`svg`](visualisation/fig9a_gdp_map_local_local_dark.svg) [`json`](visualisation/fig9a_gdp_map_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_local_dark.png) [`svg`](visualisation/fig9a_gdp_map_local_dark.svg) [`json`](visualisation/fig9a_gdp_map_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9a_gdp_map_local_local_no_branding_dark.png) [`svg`](visualisation/fig9a_gdp_map_local_local_no_branding_dark.svg) [`json`](visualisation/fig9a_gdp_map_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9a_gdp_map_local_no_branding_dark.png) [`svg`](visualisation/fig9a_gdp_map_local_no_branding_dark.svg) [`json`](visualisation/fig9a_gdp_map_local_no_branding_dark.json)   

!["fig9a_gdp_map_local"](visualisation/fig9a_gdp_map_local.svg "fig9a_gdp_map_local")

  

!["fig9a_gdp_map_local_dark"](visualisation/fig9a_gdp_map_local_dark.svg "fig9a_gdp_map_local")

## Figure 9b  

Data: [`csv`](data/fig9b_co2_map_local.csv)  
GitHub: [fig9b_co2_map_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_local.png) [`svg`](visualisation/fig9b_co2_map_local_local.svg) [`json`](visualisation/fig9b_co2_map_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_local.png) [`svg`](visualisation/fig9b_co2_map_local.svg) [`json`](visualisation/fig9b_co2_map_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_local_no_branding.png) [`svg`](visualisation/fig9b_co2_map_local_local_no_branding.svg) [`json`](visualisation/fig9b_co2_map_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_local_no_branding.png) [`svg`](visualisation/fig9b_co2_map_local_no_branding.svg) [`json`](visualisation/fig9b_co2_map_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_local_dark.png) [`svg`](visualisation/fig9b_co2_map_local_local_dark.svg) [`json`](visualisation/fig9b_co2_map_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_local_dark.png) [`svg`](visualisation/fig9b_co2_map_local_dark.svg) [`json`](visualisation/fig9b_co2_map_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9b_co2_map_local_local_no_branding_dark.png) [`svg`](visualisation/fig9b_co2_map_local_local_no_branding_dark.svg) [`json`](visualisation/fig9b_co2_map_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9b_co2_map_local_no_branding_dark.png) [`svg`](visualisation/fig9b_co2_map_local_no_branding_dark.svg) [`json`](visualisation/fig9b_co2_map_local_no_branding_dark.json)   

!["fig9b_co2_map_local"](visualisation/fig9b_co2_map_local.svg "fig9b_co2_map_local")

  

!["fig9b_co2_map_local_dark"](visualisation/fig9b_co2_map_local_dark.svg "fig9b_co2_map_local")

## Figure 9c  

Data: [`csv`](data/fig9c_gpr_map_local.csv)  
GitHub: [fig9c_gpr_map_local](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-do-greening-prosperity-stripes-help-measure-progress-towards-net-zero)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_local.png) [`svg`](visualisation/fig9c_gpr_map_local_local.svg) [`json`](visualisation/fig9c_gpr_map_local_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_local.png) [`svg`](visualisation/fig9c_gpr_map_local.svg) [`json`](visualisation/fig9c_gpr_map_local.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_local_no_branding.png) [`svg`](visualisation/fig9c_gpr_map_local_local_no_branding.svg) [`json`](visualisation/fig9c_gpr_map_local_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_local_no_branding.png) [`svg`](visualisation/fig9c_gpr_map_local_no_branding.svg) [`json`](visualisation/fig9c_gpr_map_local_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_local_dark.png) [`svg`](visualisation/fig9c_gpr_map_local_local_dark.svg) [`json`](visualisation/fig9c_gpr_map_local_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_local_dark.png) [`svg`](visualisation/fig9c_gpr_map_local_dark.svg) [`json`](visualisation/fig9c_gpr_map_local_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig9c_gpr_map_local_local_no_branding_dark.png) [`svg`](visualisation/fig9c_gpr_map_local_local_no_branding_dark.svg) [`json`](visualisation/fig9c_gpr_map_local_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig9c_gpr_map_local_no_branding_dark.png) [`svg`](visualisation/fig9c_gpr_map_local_no_branding_dark.svg) [`json`](visualisation/fig9c_gpr_map_local_no_branding_dark.json)   

!["fig9c_gpr_map_local"](visualisation/fig9c_gpr_map_local.svg "fig9c_gpr_map_local")

  

!["fig9c_gpr_map_local_dark"](visualisation/fig9c_gpr_map_local_dark.svg "fig9c_gpr_map_local")

